In [4]:
import gym
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import Model
import numpy as np
import matplotlib.pyplot as plt
import os
import json
import random
import tensorflow_probability as tfp
from tensorflow.keras import regularizers

/Users/anthonylaw/anaconda3/envs/rl_models/lib/python3.7/site-packages/pandas/compat/numpy/__init__.py:10: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  _nlv = LooseVersion(_np_version)
/Users/anthonylaw/anaconda3/envs/rl_models/lib/python3.7/site-packages/setuptools/_distutils/version.py:351: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  other = LooseVersion(other)
/Users/anthonylaw/anaconda3/envs/rl_models/lib/python3.7/site-packages/pandas/compat/numpy/function.py:120: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(__version__) >= LooseVersion("1.17.0"):
/Users/anthonylaw/anaconda3/envs/rl_models/lib/python3.7/site-packages/matplotlib/__init__.py:202: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(module.__version__) < minver:
/Users/anthonylaw/anaconda3/envs/r

### Env Setup

In [5]:
problem = "Hopper-v3"
env = gym.make(problem)

num_states = env.observation_space.shape[0]
num_actions = env.action_space.shape[0]
upper_bound = env.action_space.high[0]
lower_bound = env.action_space.low[0]

/Users/anthonylaw/anaconda3/envs/rl_models/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### Actor Model

#### Note:

Same as PPO actor. Gaussian policy

In [6]:
EPSILON = 1e-10

class Actor(Model):

    def __init__(self, action_dimensions, action_bound):
        super().__init__()
        self.action_dim, self.upper_bound = action_dimensions, action_bound
        self.sample_dist = tfp.distributions.MultivariateNormalDiag(loc=tf.zeros(self.action_dim),
                                                                    scale_diag=tf.ones(self.action_dim))
        self.dense1_layer = layers.Dense(64, activation="relu")
        self.dense2_layer = layers.Dense(64, activation="relu")
        self.mean_layer = layers.Dense(self.action_dim)
        self.stdev_layer = layers.Dense(self.action_dim)

    def call(self, state, eval_mode=False):

        a1 = self.dense1_layer(state)
        a2 = self.dense2_layer(a1)
        mu = self.mean_layer(a2)

        log_sigma = self.stdev_layer(a2)
        sigma = tf.exp(log_sigma)
        sigma = tf.clip_by_value(sigma, EPSILON, 2.718)

        dist = tfp.distributions.MultivariateNormalDiag(loc=mu, scale_diag=sigma)
        
        if eval_mode:
            action_ = mu
        else:
            action_ = tf.math.add(mu, tf.math.multiply(sigma, tf.expand_dims(self.sample_dist.sample(), 0)))
 
        action = tf.tanh(action_)

        log_pi_ = dist.log_prob(action_)     
        log_pi = log_pi_ - tf.reduce_sum(tf.math.log(tf.clip_by_value(1 - action**2, EPSILON, 1.0)), axis=1)
        
        return action*self.upper_bound, log_pi


In [7]:
actor_test = Actor(num_actions, upper_bound)

In [8]:
obs = env.reset()
obs
tf_obs = tf.expand_dims(obs, 0)
tf_obs
a_test, log_a_test = actor_test(tf_obs)
print(a_test, log_a_test)

tf.Tensor([[ 0.9941358  -0.72490025 -0.6927037 ]], shape=(1, 3), dtype=float32) tf.Tensor([-1.3114829], shape=(1,), dtype=float32)


In [9]:
actor_test.summary()

Model: "actor"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               multiple                  768       
                                                                 
 dense_1 (Dense)             multiple                  4160      
                                                                 
 dense_2 (Dense)             multiple                  195       
                                                                 
 dense_3 (Dense)             multiple                  195       
                                                                 
Total params: 5,318
Trainable params: 5,318
Non-trainable params: 0
_________________________________________________________________


### Critic Wrapper

#### Note:

Different from PPO, critic evaluate state-action pairs.

In [10]:
class Critic_Wrapper():
    def __init__(self, state_dim, action_dim):
        self.s_dim=state_dim
        self.a_dim=action_dim
        
    def get_critic(self):
        # State as input
        state_input = layers.Input(shape=(self.s_dim))
        state_out = layers.Dense(32, activation="relu")(state_input)

        # Action as input
        action_input = layers.Input(shape=(self.a_dim))
        action_out = layers.Dense(32, activation="relu")(action_input)

        # Concatenating
        concat = layers.Concatenate()([state_out, action_out])
        out = layers.Dense(64, activation="relu")(concat)
        outputs = tf.squeeze(layers.Dense(1)(out))

        # Outputs single value for give state-action
        model = tf.keras.Model([state_input, action_input], outputs)

        return model

In [11]:
critic_gen = Critic_Wrapper(num_states, num_actions)
critic_test = critic_gen.get_critic()

In [12]:
obs = env.reset()
obs

array([ 1.25089241e+00, -2.10895261e-03, -1.11322642e-03,  3.80833621e-03,
        2.32433107e-03,  3.59612832e-03, -4.67824255e-03, -4.85445677e-04,
       -1.80631925e-03,  4.81813770e-03, -2.55632145e-03])

In [13]:
tf_obs = tf.expand_dims(obs, 0)
a_test, log_a_test = actor_test(tf_obs)
tf_obs

<tf.Tensor: shape=(1, 11), dtype=float64, numpy=
array([[ 1.25089241e+00, -2.10895261e-03, -1.11322642e-03,
         3.80833621e-03,  2.32433107e-03,  3.59612832e-03,
        -4.67824255e-03, -4.85445677e-04, -1.80631925e-03,
         4.81813770e-03, -2.55632145e-03]])>

In [14]:
v_test = critic_test([tf_obs, a_test])
v_test

<tf.Tensor: shape=(), dtype=float32, numpy=-0.038194172>

In [15]:
obs_new, _, _, _ = env.step(a_test[0])
tf_obs_new = tf.expand_dims(obs_new, 0)
statex2 = tf.convert_to_tensor([obs, obs_new])
print(statex2.shape)

(2, 11)


In [16]:
a_2, loga_2 = actor_test(statex2)

print(a_2.shape, loga_2.shape)

(2, 3) (2,)


In [17]:
v_2 = critic_test([statex2, a_2])
print(v_2.shape)

(2,)


In [18]:
critic_test.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 11)]         0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 3)]          0           []                               
                                                                                                  
 dense_4 (Dense)                (None, 32)           384         ['input_1[0][0]']                
                                                                                                  
 dense_5 (Dense)                (None, 32)           128         ['input_2[0][0]']                
                                                                                              

### Replay Buffer

In [19]:
class Buffer:
    def __init__(self, obs_dim, a_dim, buffer_capacity=100000, batch_size=256):
        
        self.obs_dim = obs_dim
        self.a_dim = a_dim
        self.buffer_capacity = buffer_capacity
        self.batch_size = batch_size
        self.buffer_counter = 0

        self.state_buffer = np.zeros((self.buffer_capacity, self.obs_dim))
        self.action_buffer = np.zeros((self.buffer_capacity, self.a_dim))
        self.reward_buffer = np.zeros((self.buffer_capacity, 1))
        self.next_state_buffer = np.zeros((self.buffer_capacity, self.obs_dim))
        self.done_buffer = np.zeros((self.buffer_capacity, 1))

    def record(self, obs_tuple):
        index = self.buffer_counter % self.buffer_capacity

        self.state_buffer[index] = obs_tuple[0]
        self.action_buffer[index] = obs_tuple[1]
        self.reward_buffer[index] = obs_tuple[2]
        self.next_state_buffer[index] = obs_tuple[3]
        self.done_buffer[index] = obs_tuple[4]
        self.buffer_counter += 1
        
    def sample(self):
        record_range = min(self.buffer_counter, self.buffer_capacity)
        batch_indices = np.random.choice(record_range, self.batch_size)

        state_batch = tf.convert_to_tensor(self.state_buffer[batch_indices])
        action_batch = tf.convert_to_tensor(self.action_buffer[batch_indices])
        reward_batch = tf.squeeze(tf.convert_to_tensor(self.reward_buffer[batch_indices]))
        next_state_batch = tf.convert_to_tensor(self.next_state_buffer[batch_indices])
        done_batch = tf.squeeze(tf.convert_to_tensor(self.done_buffer[batch_indices]))
        
        return (state_batch,
               action_batch,
               reward_batch,
               next_state_batch,
               done_batch)


In [20]:
buffer1 = Buffer(num_states, num_actions, 100, 10)

In [21]:
prev_obs = env.reset()

for i in range(buffer1.buffer_capacity):
    a, _ = actor_test(tf.expand_dims(prev_obs, 0))
    obs, r, d, _ = env.step(a[0])
    
    buffer1.record((prev_obs, a[0], r, obs, d))
    
    prev_obs = obs
    

In [22]:
buffer1.sample()

(<tf.Tensor: shape=(10, 11), dtype=float64, numpy=
 array([[ 5.23663984e-01, -1.13121674e+00, -1.98751898e-01,
         -2.62706605e+00, -6.02562130e-01,  1.64063057e+00,
         -1.03479354e+00, -1.41634745e+00, -1.89422938e+00,
          1.98933195e-01, -6.94132881e+00],
        [ 1.08457497e+00, -5.65752572e-01, -1.66194852e-02,
         -1.41946917e+00,  7.53706495e-01, -1.63635638e-01,
         -1.95810178e+00, -6.61124435e-01,  3.90131515e+00,
         -1.00000000e+01, -3.92664112e+00],
        [ 1.19934065e+00, -1.96396271e-01,  4.72482934e-03,
         -4.74358212e-01,  4.98837054e-01, -3.08678026e-02,
         -1.65350566e-01, -2.88059803e+00, -1.09175940e+00,
         -5.26309694e+00,  7.67004363e+00],
        [ 4.44129089e-01, -1.16710359e+00, -4.19732410e-01,
         -2.60585857e+00, -8.04937074e-01,  2.90345238e-01,
          2.68244490e-02, -2.10895590e+00, -3.10609059e+00,
          9.71435185e-01,  8.70259724e-01],
        [ 1.25295866e+00,  2.54342752e-03,  2.8212076

### Soft Actor Critic

In [37]:
class SAC:
    
    def __init__(self, env, observation_dimensions, action_dimensions, action_bound, buffer_capacity,
                 minibatch_size=256, gamma=0.99, tau=0.95, lr=3e-4):
        
        self.env = env
        
        self.a = Actor(action_dimensions, action_bound)
        self.c_gen = Critic_Wrapper(observation_dimensions, action_dimensions)
        self.c1 = self.c_gen.get_critic()
        self.c2 = self.c_gen.get_critic()
        self.tc1 = self.c_gen.get_critic()
        self.tc2 = self.c_gen.get_critic()
        
        self.tc1.set_weights(self.c1.get_weights())
        self.tc2.set_weights(self.c2.get_weights())

        self.te = -np.prod(action_dimensions)
        self.alpha = tf.Variable(0.0, dtype=tf.float32)
        
        self.a_opt = tf.keras.optimizers.Adam(learning_rate=lr)
        self.c1_opt = tf.keras.optimizers.Adam(learning_rate=lr)
        self.c2_opt = tf.keras.optimizers.Adam(learning_rate=lr)                                                  
        self.alpha_opt = tf.keras.optimizers.Adam(learning_rate=lr)   
        
        self.buffer = Buffer(observation_dimensions, action_dimensions, buffer_capacity, minibatch_size)
        
        self.gamma, self.tau = gamma, tau
        
    def train(self, max_env_step):
        t = 0
        epo = 0
        while t < max_env_step:
            p_s = self.env.reset()
            a_losses = []
            c1_losses = []
            c2_losses = []
            alpha_losses = []
            while True:
                a, log_a = self.a(tf.expand_dims(p_s, 0))
                a=a[0]
                s, r, d, _ = self.env.step(a)
                end = 0 if d else 1
                
                self.buffer.record((p_s, a, r, s, end))
                data = self.buffer.sample()
                
                a_loss, c1_loss, c2_loss, alpha_loss = self.update(data)
                
                a_losses.append(a_loss.numpy())
                c1_losses.append(c1_loss.numpy())
                c2_losses.append(c2_loss.numpy())
                alpha_losses.append(alpha_loss.numpy())
                
                t = t+1
                
                if d:
                    break
                p_s = s
                
            print("Epoch {:04d}".format(epo), "Policy Avg. Loss: ", np.mean(a_losses), 
                  ", Critic 1 Avg. Loss: ",  np.mean(c1_losses), 
                  ", Critic 2 Avg. Loss: ",  np.mean(c2_losses), 
                  ", Alpha 1 Avg. Loss: ",  np.mean(alpha_losses), flush=True)
            epo = epo+1


    @tf.function
    def update(self, data):
        s_b, a_b, r_b, ns_b, d_b = data
        with tf.GradientTape() as tape_c1, tf.GradientTape() as tape_c2:
            q1 = self.c1([s_b, a_b])
            q2 = self.c2([s_b, a_b])
            na, nlog_a = self.a(ns_b)
            
            tq1 = self.tc1([ns_b, na])
            tq2 = self.tc2([ns_b, na])
            
            min_qt = tf.math.minimum(tq1,tq2)
            
            soft_qt = min_qt - (self.alpha*nlog_a)
            
            y = tf.stop_gradient(r_b+self.gamma*d_b*tf.cast(soft_qt, dtype=tf.float64))
            
            L_c1 = 0.5*tf.reduce_mean((y-tf.cast(q1, dtype=tf.float64))**2)
            L_c2 = 0.5*tf.reduce_mean((y-tf.cast(q2, dtype=tf.float64))**2)
        c1_grad = tape_c1.gradient(L_c1, self.c1.trainable_variables)
        c2_grad = tape_c2.gradient(L_c2, self.c2.trainable_variables)
        
        self.c1_opt.apply_gradients(zip(c1_grad, self.c1.trainable_variables))
        self.c2_opt.apply_gradients(zip(c2_grad, self.c2.trainable_variables))
        
        for (tc1w, c1w) in zip(self.tc1.variables, self.c1.variables):
            tc1w.assign(tc1w*self.tau + c1w*(1.0-self.tau))
        for (tc2w, c2w) in zip(self.tc2.variables, self.c2.variables):
            tc2w.assign(tc2w*self.tau + c2w*(1.0-self.tau))
            
        with tf.GradientTape() as tape_a, tf.GradientTape() as tape_alpha:
            a, log_a = self.a(s_b)
            qa1 = self.c1([s_b, a])
            qa2 = self.c2([s_b, a])
            
            soft_qa = tf.reduce_mean([qa1,qa2], axis=0)
            
            L_a = -tf.reduce_mean(soft_qa-self.alpha*log_a)
            L_alpha = -tf.reduce_mean(self.alpha*tf.stop_gradient(log_a + self.te))
        grad_a = tape_a.gradient(L_a, self.a.trainable_variables)
        grad_alpha = tape_alpha.gradient(L_alpha, [self.alpha])
        self.a_opt.apply_gradients(zip(grad_a, self.a.trainable_variables))
        self.alpha_opt.apply_gradients(zip(grad_alpha, [self.alpha]))
        
        return L_a, L_c1, L_c2, L_alpha
    
    def save_weights(self, dir_path):
        self.a.save_weights(dir_path+"/a.ckpt")
        print("Saved actor weights", flush=True)
        self.c1.save_weights(dir_path+"/c1.ckpt")
        print("Saved critic 1 weights", flush=True)
        self.c2.save_weights(dir_path+"/c2.ckpt")
        print("Saved critic 2 weights", flush=True)

    def load_weights(self, dir_path):
        try:
            self.a.load_weights(dir_path+"/a.ckpt")
            print("Loaded actor weights", flush=True)
            self.c1.load_weights(dir_path+"/c1.ckpt")
            print("Loaded critic 1 weights", flush=True)
            self.c2.load_weights(dir_path+"/c2.ckpt")
            print("Loaded critic 2 weights", flush=True)
        except ValueError:
            print("ERROR: Please make sure weights are saved as .ckpt", flush=True)
            
    def eval_rollout(self, eval_env):
        eps_r = 0
        eval_obs = eval_env.reset()
        
        while True:
            eval_env.render()

            tf_eval_obs = tf.expand_dims(tf.convert_to_tensor(eval_obs), 0)

            eval_a, eval_log_a = self.a(tf_eval_obs, eval_mode=True)

            eval_a = eval_a[0]

            eval_obs_new, eval_r, eval_d, _ = eval_env.step(eval_a)

            eps_r += eval_r

            if eval_d:
                break
                
            eval_obs = eval_obs_new
        eval_env.close()
        print("rollout episodic reward: ", eps_r, flush=True)


In [38]:
sac1 = SAC(env, num_states, num_actions, upper_bound, 1000000)

In [39]:
sac1.train(1000)

Epoch 0000 Policy Avg. Loss:  -0.07243134 , Critic 1 Avg. Loss:  0.22062063767155676 , Critic 2 Avg. Loss:  0.3344533875284732 , Alpha 1 Avg. Loss:  -0.0038230228
Epoch 0001 Policy Avg. Loss:  -0.2707948 , Critic 1 Avg. Loss:  0.15812808112825003 , Critic 2 Avg. Loss:  0.17292359268355023 , Alpha 1 Avg. Loss:  -0.011478158
Epoch 0002 Policy Avg. Loss:  -0.3884465 , Critic 1 Avg. Loss:  0.1412082966394509 , Critic 2 Avg. Loss:  0.12594620861978278 , Alpha 1 Avg. Loss:  -0.018301968
Epoch 0003 Policy Avg. Loss:  -0.40722534 , Critic 1 Avg. Loss:  0.13687380721694256 , Critic 2 Avg. Loss:  0.14382856831562876 , Alpha 1 Avg. Loss:  -0.036046352
Epoch 0004 Policy Avg. Loss:  -0.5636535 , Critic 1 Avg. Loss:  0.13242284611894808 , Critic 2 Avg. Loss:  0.20022134601997435 , Alpha 1 Avg. Loss:  -0.031472337
Epoch 0005 Policy Avg. Loss:  -0.6606414 , Critic 1 Avg. Loss:  0.13944941201231384 , Critic 2 Avg. Loss:  0.15885819223142922 , Alpha 1 Avg. Loss:  -0.033156212
Epoch 0006 Policy Avg. Loss

In [40]:
sac1.save_weights("/Users/anthonylaw/Desktop/Work_Research/rl_models/MA2C/sac/devel/test_out/weights")

Saved actor weights
Saved critic 1 weights
Saved critic 2 weights


In [41]:
sac2 = SAC(env, num_states, num_actions, upper_bound, 1000000)

In [42]:
sac2.load_weights("/Users/anthonylaw/Desktop/Work_Research/rl_models/MA2C/sac/devel/test_out/weights")

Saved actor weights
Saved critic 1 weights
Saved critic 2 weights


In [43]:
sac2.train(500)

Epoch 0000 Policy Avg. Loss:  -22.552607 , Critic 1 Avg. Loss:  133.49638950523433 , Critic 2 Avg. Loss:  129.55168005371527 , Alpha 1 Avg. Loss:  -0.0071828915
Epoch 0001 Policy Avg. Loss:  -18.767363 , Critic 1 Avg. Loss:  7.316584747076681 , Critic 2 Avg. Loss:  5.795925751358901 , Alpha 1 Avg. Loss:  -0.05522415
Epoch 0002 Policy Avg. Loss:  -17.96469 , Critic 1 Avg. Loss:  5.507646484330524 , Critic 2 Avg. Loss:  5.2844075638829064 , Alpha 1 Avg. Loss:  -0.1362014
Epoch 0003 Policy Avg. Loss:  -18.942528 , Critic 1 Avg. Loss:  5.564156786934953 , Critic 2 Avg. Loss:  5.366625323636581 , Alpha 1 Avg. Loss:  -0.24017943
Epoch 0004 Policy Avg. Loss:  -20.729391 , Critic 1 Avg. Loss:  6.5831684525339575 , Critic 2 Avg. Loss:  6.137281363338495 , Alpha 1 Avg. Loss:  -0.35341176
Epoch 0005 Policy Avg. Loss:  -22.160152 , Critic 1 Avg. Loss:  6.340519342927752 , Critic 2 Avg. Loss:  5.843234969346588 , Alpha 1 Avg. Loss:  -0.30649784
Epoch 0006 Policy Avg. Loss:  -23.273882 , Critic 1 Av